In [61]:
#!/usr/bin/env python
# encoding: utf-8

import sys
import os
import math
import pprint
import json
import sqlite3
import time
import datetime
import numpy as np
from os import listdir
from os.path import isfile, join

#dbpath = '/Users/mszell/data/githubarchive/derived/'
#dbname = 'githubarchive.db'
#datapath = '/Users/mszell/data/githubarchive/raw/'

dbpath = ''
dbname = 'REPO_TEST55.sqlite'
datapath = os.getcwd()


In [44]:
def date_from_filename(fname):
    dt = datetime.datetime.strptime(fname[len('githubarchive_'):-len('.json')], '%Y-%m-%d')
    return dt.strftime('%Y-%m-%d')

In [45]:
def date_from_json(creation_date):
    dt = datetime.datetime.strptime(creation_date[:19], '%Y-%m-%dT%H:%M:%S')
    return dt

In [47]:
def parse_popularity(jsonline):
    if jsonline.get('repository'):
        repoid = jsonline['repository'].get('id')
        forks = jsonline['repository'].get('forks')
        watchers = jsonline['repository'].get('watchers')
        stargazers = jsonline['repository'].get('stargazers')
        size = jsonline['repository'].get('size')
        date = date_from_json(jsonline['repository']['pushed_at'])
        owner = jsonline['repository']['owner']
        return repoid, forks, watchers, stargazers, size, date, owner
    elif jsonline.get('type')=='PullRequestEvent':
        if jsonline.get('repo') and jsonline['payload']['pull_request'].get('base') and jsonline['payload']['pull_request']['base'].get('repo'):
            repoid = jsonline['repo'].get('id')
            forks = jsonline['payload']['pull_request']['base']['repo'].get('forks_count')
            watchers = jsonline['payload']['pull_request']['base']['repo'].get('watchers_count')
            stargazers = jsonline['payload']['pull_request']['base']['repo'].get('stargazers_count')
            size = jsonline['payload']['pull_request']['base']['repo'].get('size')
            date = date_from_json(jsonline['created_at'])
            owner = jsonline['payload']['pull_request']['base']['repo']['owner'].get('login')
            return repoid, forks, watchers, stargazers, size, date, owner

In [48]:
def parse_user(line):
    if line.get('actor_attributes') and line['actor_attributes'].get('login') and line['actor_attributes'].get('name'):
        login=line['actor_attributes']['login']
        name=line['actor_attributes']['name']
        if line['actor_attributes'].get('email'):
            e_mail=line['actor_attributes']['email']
        else:
            e_mail=np.nan
        return login,name,e_mail
    elif line['type']=='PushEvent' and type(line['actor'])==dict and line['actor'].get('login') and line.get('payload') and line['payload'].get('shas') and len(line['payload']['shas'])==4:
        login=jsonline['actor']['login']
        y=line['payload'].get('shas')[0]
        name=y[-1]
        e_mail=y[1]
        return login,name,e_mail
    elif line['type']=='PushEvent' and type(line['actor'])!=dict and line.get('payload') and line['payload'].get('shas') and len(line['payload']['shas'])==4:
        login=jsonline['actor']
        y=line['payload'].get('shas')[0]
        name=y[-1]
        e_mail=y[1]
        return login,name,e_mail
    elif line['type']=='PushEvent' and type(line['actor'])==dict and line['actor'].get('login') and line.get('payload') and line['payload'].get('commits') and line['payload']['commits'][0].get('author'):
        login=line['actor']['login']
        y=line['payload']['commits'][0]['author']
        name=y.get('name')
        e_mail=y.get('email')
        return login,name,e_mail

In [49]:
def parse_create_event(event):
    if event['payload'].get('object')=='repository' and event['repo'].get('id') and event['payload'].get('name') and event.get('actor').get('login'):
        repo_name=event['payload'].get('name')
        repoid=event['repo']['id']
        created_at=date_from_json(event.get('created_at'))
        user=event['actor']['login']
        description='NA'
        #print repoid, repo_name, user, created_at, description
        return repoid, repo_name, user, created_at, description
    elif event.get('payload') and event['payload'].get('ref_type')=='repository' and event.get('repository') and event.get('actor'):
        repo_name=event['repository']['name']
        repoid=event['repository']['id']
        created_at=date_from_json(event['repository'].get('created_at'))
        user=event['actor']
        description=event['payload'].get('description')
        return repoid, repo_name, user, created_at, description
    elif event.get('payload') and event['payload'].get('ref_type')=='repository' and event.get('repo') and event.get('actor'):
        repo_name=event['repo']['name']
        repoid=event['repo']['id']
        created_at=date_from_json(event.get('created_at'))
        user=event['actor']['login']
        description=event['payload'].get('description')
        return repoid, repo_name, user, created_at, description

In [50]:
def parse(event):
    #print event['type']
    if type(event)==dict:
        if event.get('repo') and event['repo'].get('id'):
            d=date_from_json(event.get('created_at'))
            if event.get('user'):
                if event['user']['type']=='User':
                    u=event['user']['login']
                    p=event['repo']['id']
                    return (u, p, d)
            elif event.get('actor'):
                if 'login' in event['actor'].keys():
                    u=event['actor']['login']
                    p=event['repo']['id']
                    return (u, p, d)
                elif event.get('payload') and event['payload'].get('actor'):
                    u=event['payload']['actor']
                    p=event['repo']['id']
                    return (u, p, d)
        elif event.get('repository'):
            u=event['actor']
            p=event['repository']['id']
            d=date_from_json(event.get('created_at'))
            return (u, p, d)

In [51]:
def parse_pullrequest_opened(event):
    if event.get('repo') and event['actor'].get('login'):
        pullreq_id=event['id']
        repoid=event['repo']['id']
        user=event['actor']['login']
        if event['payload']['action']=='opened':
            opened_at=date_from_json(event['created_at'])
        return (pullreq_id, repoid, user, opened_at)
    elif event.get('repository') and event['payload'].get('actor'):
        pullreq_id=event['id']
        repoid=event['repository']['id']
        user=event['payload']['actor']
        opened_at=date_from_json(event.get('created_at'))
        return (pullreq_id, repoid, user, opened_at)
    elif event.get('repository'):
        pullreq_id=event['payload']['pull_request']['id']
        repoid=event['repository']['id']
        user=event['actor']
        opened_at=date_from_json(event.get('created_at'))
        return (pullreq_id, repoid, user, opened_at)

In [52]:
def parse_pullrequest_closed(event):
    if event.get('repo') and event['actor'].get('login'):
        pullreq_id=event['id']
        repoid=event['repo']['id']
        user=event['actor']['login']
        closed_at=date_from_json(event['created_at'])
        return (pullreq_id, repoid, user, closed_at)
    elif event.get('repository') and event['payload'].get('actor'):
        pullreq_id=event['id']
        repoid=event['repository']['id']
        user=event['payload']['actor']
        closed_at=date_from_json(event['payload']['pull_request'].get('closed_at'))
        return (pullreq_id, repoid, user, closed_at)
    elif event.get('repository'):
        pullreq_id=event['payload']['pull_request']['id']
        repoid=event['repository']['id']
        user=event['actor']
        closed_at=date_from_json(event['payload']['pull_request'].get('closed_at'))
        return (pullreq_id, repoid, user, closed_at)

In [53]:
def rep(jsonline):
    if jsonline.get("type") == "CreateEvent" and jsonline.get('payload').get('object')=='repository':
        repoid, repo_name, user, created_at, description=parse_create_event(jsonline)
        return  repoid, repo_name, user, created_at, description
    elif jsonline.get("type") == "CreateEvent" and jsonline.get('payload').get('ref_type')=='repository':
        repoid, repo_name, user, created_at, description=parse_create_event(jsonline)
        return  repoid, repo_name, user, created_at, description

In [54]:
def parse_pullrequest_merged(event):
    if event.get('repository') and event['payload'].get('actor'):
        pullreq_id=event['id']
        repoid=event['repository']['id']
        user=event['payload']['actor']
        merged_at=date_from_json(event['payload']['pull_request'].get('closed_at'))
        #print pullreq_id, repoid, user, merged_at
        return (pullreq_id, repoid, user, merged_at)
    elif event.get('repo'):
        pullreq_id=event['payload']['pull_request']['id']
        repoid=event['repo']['id']
        user=event['actor']['login']
        merged_at=date_from_json(event['payload']['pull_request'].get('closed_at'))
        #print pullreq_id, repoid, user, merged_at
        return (pullreq_id, repoid, user, merged_at)
    elif event.get('repository') and event.get('actor_attributes'):
        pullreq_id=event['payload']['pull_request']['id']
        repoid=event['repository']['id']
        user=event['actor_attributes']['login']
        merged_at=date_from_json(event['payload']['pull_request'].get('closed_at'))
        return (pullreq_id, repoid, user, merged_at)

In [55]:
#MembershipEvent
def parse_membership_event(event):
    user=event['actor']['login']
    org=event['org']['login']
    action=event['payload']['action']
    return (user,org,action) 

In [56]:
def parse_follow_event(event):
    if type(event['actor'])==dict:
        follower=event['payload']['actor']
        target=event['payload']['target']['login']
    else:
        follower=event['actor']
        target=event['payload']['target']['login']
    return (follower,target)

In [57]:
#MemberEvent
def parse_member_event(event):
    if 'repo' not in event.keys():
        user=event['actor']
        repo=event['repository']['id']
        owner=event['repository']['owner']
        action=event['payload']['action']
        return (repo,user,owner,action)
    elif event['payload'].get('member') and type(event['payload']['member'])==dict:
        user=event['payload']['member']['login']
        repo=event['repo']['id']
        owner=event['actor']['login']
        action=event['payload']['action']
        return (repo,user,owner,action)
    else:
        user=event['payload']['member']
        repo=event['repo']['id']
        owner=event['payload']['actor']
        action=event['payload']['action']
        return (repo,user,owner,action)         

In [62]:
conn = sqlite3.connect(dbpath + dbname)
c = conn.cursor()
c.execute('''DROP TABLE IF EXISTS repository''')
c.execute('''CREATE TABLE IF NOT EXISTS repository
                    (repositoryid INTEGER PRIMARY KEY, repositoryname TEXT, userid TEXT, created_at DATE, description TEXT)''')
c.execute('''DROP TABLE IF EXISTS users''')
c.execute('''CREATE TABLE IF NOT EXISTS users
                    (userlogin TEXT PRIMARY KEY, username TEXT, useremail TEXT)''')

c.execute('''DROP TABLE IF EXISTS push''')
c.execute('''CREATE TABLE IF NOT EXISTS push
                 (repositoryid INTEGER, pushed_at DATE, userlogin TEXT)''')

c.execute('''DROP TABLE IF EXISTS watch''')
c.execute('''CREATE TABLE IF NOT EXISTS watch
                 (repositoryid INTEGER, watched_at DATE, byuserlogin TEXT)''')

c.execute('''DROP TABLE IF EXISTS pull_request_opened''')
c.execute('''CREATE TABLE IF NOT EXISTS pull_request_opened
                 (pulrrequestid INTEGER PRIMARY KEY, repositoryid INTEGER, userlogin TEXT, opened_at DATE)''')

c.execute('''DROP TABLE IF EXISTS pull_request_closed''')
c.execute('''CREATE TABLE IF NOT EXISTS pull_request_closed
                 (pulrrequestid INTEGER PRIMARY KEY, repositoryid INTEGER, userlogin TEXT, closed_at DATE)''')

c.execute('''DROP TABLE IF EXISTS pull_request_merged''')
c.execute('''CREATE TABLE IF NOT EXISTS pull_request_merged
                 (pulrrequestid INTEGER PRIMARY KEY, repositoryid INTEGER, userlogin TEXT, merged_at DATE)''')

c.execute('''DROP TABLE IF EXISTS fork''')
c.execute('''CREATE TABLE IF NOT EXISTS fork
                 (targetrepoid INTEGER, forked_at DATE,  byuserlogin TEXT)''')

c.execute('''DROP TABLE IF EXISTS orgmembership''')
c.execute('''CREATE TABLE IF NOT EXISTS orgmembership
                 (added_at DATE, targetuserlogin TEXT, orglogin TEXT, action TEXT )''')

c.execute('''DROP TABLE IF EXISTS memberadd''')
c.execute('''CREATE TABLE IF NOT EXISTS memberadd
                 (targetrepoid INTEGER, added_at DATE, targetuserlogin TEXT, owneruserlogin TEXT, action TEXT )''')

c.execute('''DROP TABLE IF EXISTS follow''')
c.execute('''CREATE TABLE IF NOT EXISTS follow
                 (added_at DATE, followerlogin TEXT, targetuserlogin TEXT )''')

c.execute('''DROP TABLE IF EXISTS success''')
c.execute('''CREATE TABLE IF NOT EXISTS success
                 (repositoryid INTEGER, forks INTEGER, watchers INTEGER, stargazers INTEGER, size INTEGER, date DATE, owner TEXT )''')

conn.commit()
conn.close()
filenames=sorted([i for i in os.listdir(datapath) if i.startswith("githubarchive_") and i.endswith('json')])
#filenames = [ f for f in listdir(datapath) if isfile(join(datapath,f)) ]
for filename in filenames[:3]:
    with open(filename) as jsonfile:
        conn = sqlite3.connect(dbpath + dbname)
        c = conn.cursor()
        i = 0
        k = 0
        p = 0
        w = 0
        pro = 0
        prc = 0
        prm = 0
        fork = 0
        fol = 0
        memb = 0
        mema = 0
        suc = 0
        for line in jsonfile:
            try:
                jsonline = json.loads(unicode(line, errors='ignore').strip())
                try:
                    repoid, repo_name, user, created_at, description=rep(jsonline)
                    query = 'INSERT OR REPLACE INTO repository VALUES (?,?,?,?,?)'
                    c.execute(query, (repoid, repo_name, user, created_at, description))
                    i = i+1
                    #print i
                    #conn.commit()
                except:
                    pass
                try:
                    login,name,e_mail=parse_user(jsonline)
                    query2 = 'INSERT OR REPLACE INTO users VALUES (?,?,?)'
                    c.execute(query2, (login,name,e_mail))
                    k = k+1
                except:
                    pass
                if jsonline.get("type") == "PushEvent":
                    try:
                        user, project, date=parse(jsonline)
                        queryp = 'INSERT OR REPLACE INTO push VALUES (?,?,?)'
                        c.execute(queryp, (project, date, user))
                        p = p+1
                    except: #repoid or user id missing
                        pass
                if jsonline.get("type") == "WatchEvent":
                        try:
                            user, project, date=parse(jsonline)
                            queryw = 'INSERT OR REPLACE INTO watch VALUES (?,?,?)'
                            c.execute(queryw, (project, date, user))
                            w = w+1
                        except: #repoid or user id missing
                            pass
                if jsonline.get("type") == "PullRequestEvent" and jsonline['payload']['action']=='opened':
                        try:
                            pullreq_id, repoid, user, opened_at=parse_pullrequest_opened(jsonline)
                            querypro = 'INSERT INTO pull_request_opened VALUES (?,?,?,?)'
                            c.execute(querypro, (pullreq_id, repoid, user, opened_at))
                            pro = pro+1
                        except:
                            pass
                if jsonline.get("type") == "PullRequestEvent" and jsonline['payload']['action']=='closed':
                        try:
                            pullreq_id, repoid, user, closed_at=parse_pullrequest_closed(jsonline)
                            queryprc = 'INSERT INTO pull_request_closed VALUES (?,?,?,?)'
                            c.execute(queryprc, (pullreq_id, repoid, user, closed_at))
                            prc = prc+1
                        except: #repoid or user id missing
                            pass
                if jsonline.get("type") == "PullRequestEvent" and jsonline['payload']['pull_request'].get('merged')==True:
                        try:
                            pullreq_id, repoid, user, merged_at=parse_pullrequest_merged(jsonline)
                            queryprm = 'INSERT INTO pull_request_merged VALUES (?,?,?,?)'
                            c.execute(queryprm, (pullreq_id, repoid, user, merged_at))
                            prm = prm+1
                        except: #repoid or user id missing
                            pass
                if jsonline.get("type") == "ForkEvent":
                        try:
                            user, project, date=parse(jsonline)
                            queryfork = 'INSERT OR REPLACE INTO fork VALUES (?,?,?)'
                            c.execute(queryfork, (project, date, user))
                            fork = fork+1
                        except:
                            pass
                if jsonline.get("type") == "MemberEvent":
                            try:
                                repo,user,owner,action=parse_member_event(jsonline)
                                querymema = 'INSERT OR REPLACE INTO memberadd VALUES (?,?,?,?,?)'
                                c.execute(querymema, (repo, date, user, owner, action))
                                mema = mema+1
                            except: #repoid or user id missing
                                pass
                if jsonline.get("type") == "FollowEvent":
                            try:
                                follower,target=parse_follow_event(jsonline)
                                queryfol = 'INSERT OR REPLACE INTO follow VALUES (?,?,?)'
                                c.execute(queryfol, (date, follower, target))
                                fol = fol+1
                            except:
                                pass
                try:
                    repoid, forks, watchers, stargazers, size, date, owner=parse_popularity(jsonline)
                    querysuc = 'INSERT OR REPLACE INTO success VALUES (?,?,?,?,?,?,?)'
                    c.execute(querysuc, (repoid, forks, watchers, stargazers, size, date, owner))
                    suc = suc+1
                except:
                    pass
            except:
                for k in ['{"repo"'+l for l in line.split('{"repo"')[1:]]:
                    jsonline=json.loads(unicode(k, errors='ignore').strip())
                    try:
                        repoid, repo_name, user, created_at, description=rep(jsonline)
                        query = 'INSERT OR REPLACE INTO repository VALUES (?,?,?,?,?)'
                        c.execute(query, (repoid, repo_name, user, created_at, description))
                        i = i+1
                        #conn.commit()
                    except:
                        pass
                    try:
                        user, project, date=parse(jsonline)
                        queryp = 'INSERT OR REPLACE INTO push VALUES (?,?,?)'
                        c.execute(queryp, (project, date, user))
                        p = p+1
                    except: #repoid or user id missing
                        pass
                    if jsonline.get("type") == "PushEvent":
                        try:
                            login,name,e_mail=parse_user(jsonline)
                            query2 = 'INSERT OR REPLACE INTO users VALUES (?,?,?)'
                            c.execute(query2, (login,name,e_mail))
                            k = k+1
                        except:
                            pass
                    if jsonline.get("type") == "WatchEvent":
                        try:
                            user, project, date=parse(jsonline)
                            queryw = 'INSERT OR REPLACE INTO watch VALUES (?,?,?)'
                            c.execute(queryw, (project, date, user))
                            w = w+1
                        except: #repoid or user id missing
                            pass
                    if jsonline.get("type") == "PullRequestEvent" and jsonline['payload']['action']=='opened':
                        try:
                            pullreq_id, repoid, user, opened_at=parse_pullrequest_opened(jsonline)
                            querypro = 'INSERT INTO pull_request_opened VALUES (?,?,?,?)'
                            c.execute(querypro, (pullreq_id, repoid, user, opened_at))
                            pro = pro+1
                        except:
                            pass
                    if jsonline.get("type") == "PullRequestEvent" and jsonline['payload']['action']=='closed':
                        try:
                            pullreq_id, repoid, user, closed_at=parse_pullrequest_closed(jsonline)
                            queryprc = 'INSERT INTO pull_request_closed VALUES (?,?,?,?)'
                            c.execute(queryprc, (pullreq_id, repoid, user, closed_at))
                            prc = prc+1
                        except: #repoid or user id missing
                            pass
                    if jsonline.get("type") == "PullRequestEvent" and jsonline['payload']['pull_request'].get('merged')==True:
                        try:
                            pullreq_id, repoid, user, merged_at=parse_pullrequest_merged(jsonline)
                            queryprm = 'INSERT INTO pull_request_merged VALUES (?,?,?,?)'
                            c.execute(queryprm, (pullreq_id, repoid, user, merged_at))
                            prm = prm+1
                        except: #repoid or user id missing
                            pass
                    if jsonline.get("type") == "ForkEvent":
                        try:
                            user, project, date=parse(jsonline)
                            queryfork = 'INSERT OR REPLACE INTO fork VALUES (?,?,?)'
                            c.execute(queryfork, (project, date, user))
                            fork = fork+1
                        except:
                            pass
                    if jsonline.get("type") == "MemberEvent":
                            try:
                                repo,user,owner,action=parse_member_event(jsonline)
                                querymema = 'INSERT OR REPLACE INTO memberadd VALUES (?,?,?,?,?)'
                                c.execute(querymema, (repo, date, user, owner, action))
                                mema = mema+1
                            except: #repoid or user id missing
                                pass
                    if jsonline.get("type") == "FollowEvent":
                            try:
                                follower,target=parse_follow_event(jsonline)
                                queryfol = 'INSERT OR REPLACE INTO follow VALUES (?,?,?)'
                                c.execute(queryfol, (date, follower, target))
                                fol = fol+1
                            except:
                                pass
                    try:
                        repoid, forks, watchers, stargazers, size, date, owner=parse_popularity(jsonline)
                        querysuc = 'INSERT OR REPLACE INTO success VALUES (?,?,?,?,?,?,?)'
                        c.execute(querysuc, (repoid, forks, watchers, stargazers, size, date, owner))
                        suc = suc+1
                    except:
                        pass

        print "Done file " + filename + ". Found and inserted " + str(i) + " valid CreationEvents."
        print "Done file " + filename + ". Found and inserted " + str(k) + " valid User Entities."
        print "Done file " + filename + ". Found and inserted " + str(p) + " valid PushEvents."
        print "Done file " + filename + ". Found and inserted " + str(w) + " valid WatchEvents."
        print "Done file " + filename + ". Found and inserted " + str(pro) + " valid PullRequestOpenEvents."
        print "Done file " + filename + ". Found and inserted " + str(prc) + " valid PullRequestClosedEvents."
        print "Done file " + filename + ". Found and inserted " + str(prm) + " valid PullRequestMergedEvents."
        print "Done file " + filename + ". Found and inserted " + str(fork) + " valid ForkEvents."
        #print "Done file " + filename + ". Found and inserted " + str(memb) + " valid MembershipEvents."
        print "Done file " + filename + ". Found and inserted " + str(mema) + " valid MemberAddEvents."
        print "Done file " + filename + ". Found and inserted " + str(mema) + " valid FollowEvents."
        print "Done file " + filename + ". Found and inserted " + str(suc) + " valid SuccessEntity."
        conn.commit()
        conn.close()

Done file githubarchive_2011-02-12.json. Found and inserted 1204 valid CreationEvents.
Done file githubarchive_2011-02-12.json. Found and inserted 394 valid User Entities.
Done file githubarchive_2011-02-12.json. Found and inserted 15378 valid PushEvents.
Done file githubarchive_2011-02-12.json. Found and inserted 3459 valid WatchEvents.
Done file githubarchive_2011-02-12.json. Found and inserted 394 valid PullRequestOpenEvents.
Done file githubarchive_2011-02-12.json. Found and inserted 386 valid PullRequestClosedEvents.
Done file githubarchive_2011-02-12.json. Found and inserted 0 valid PullRequestMergedEvents.
Done file githubarchive_2011-02-12.json. Found and inserted 877 valid ForkEvents.
Done file githubarchive_2011-02-12.json. Found and inserted 198 valid MemberAddEvents.
Done file githubarchive_2011-02-12.json. Found and inserted 198 valid FollowEvents.
Done file githubarchive_2011-02-12.json. Found and inserted 0 valid SuccessEntity.
Done file githubarchive_2013-02-27.json. Fo

## TODO

- see parsing differences over years
- run NLP
- build networks by years
- homophily